# 下载数据集

**数据集内容：**

包含text，category和output，任务是根据text的内容在category的list中选取合适的词语作为output

**数据集地址：** https://modelscope.cn/datasets/swift/zh_cls_fudan-news/files

# 数据预处理

使用的模型是Qwen2-Instruct，是生成式语言模型 **（Decoder-only的自回归生成）**，不同于传统分类网络只接收固定的维度向量，这里需要把文本+选项作为模型的输入，答案作为输出，通过prompt将任务导向为分类，但是 **本质上还是生产式语言模型** ，只不过通过训练数据和prompt让结果落在固定的结合内 **(限制答案生成空间)**。

In [13]:
# 先查看原始数据的格式
import json
import os

# windows系统的相对路径相较于linux比较麻烦
file_path = os.path.join(os.getcwd(), "dataset", "train.jsonl")
with open(file_path, "r") as f:
    for line in f:
        data = json.loads(line)
        print(data)
        break

{'text': '【 文献号 】4-2054\t【原文出处】史学月刊\t【原刊地名】开封\t【原刊期号】199804\t【原刊页号】118～120\t【分 类 号】K5\t【分 类 名】世界史\t【复印期号】199809\t【 标  题 】正确评价世界近代史上的历史人物\t【 作  者 】秦元春\t【作者简介】作者秦元春，1964年生，淮南师范专科学校历史系讲师。淮南，232001\t【 正  文 】\t历史科学是一门准确而严谨的科学。正确了解和评价各个时期的历史人物及其所从事的活动，是历史研究的一个重要组成部分。在世界历史的长河中，有许多类别不同的历史人物，由于特定的条件和环境，具有比较复杂的特性。因此，遵循马克思历史唯物主义的观点，正确评价历史人物，在历史研究与教学过程中尤为重要。一  正确评价资产阶级革命中的各种头面人物世界近代史是资本主义发生、发展的历史。资本主义社会如同封建社会一样，都是人类社会历史发展过程中的必然阶段。17世纪至18世纪欧美发生的早期资产阶级革命，用资本主义制度战胜了封建制度，显示了资本主义制度比封建制度的巨大优越性。当时，最先出来领导革命并执掌政权的总是上层金融资产阶级。这是不足为怪的。因为它是最有经济实力、政治上最成熟的阶层，由它建立统治权在客观上是历史的必然。因此，正确评价早期资产阶级革命中具有开拓性的历史人物，必须根据当时时代的特点。《共产党宣言》中说：“资产阶级在历史上曾经起过非常革命的作用。”马克思的这一观点，尤应引起我们的重视。如何评价克伦威尔？首先，克伦威尔是一个处于世界从封建社会开始向资本主义社会转变的历史时期的人物，他在当时是能够顺应世界从封建社会向资本主义社会转变这一历史潮流的。他的历史任务就是在英国推翻封建制度，建立资本主义制度。克伦威尔基本上完成了历史所赋予的时代任务。在革命一开始，他就参加了反对斯图亚特王朝的斗争；后来，在审讯查理一世的过程中态度坚决，他又始终参加了对国王的审判。正是由于他的坚持，国王才被推上断头台，英国才宣布为“共和国和自由国家”。无疑，克伦威尔促进了英国社会的变革与向前发展。其次，克伦威尔领导了多次重大战役，具有卓越的军事才能。克伦威尔不愧为英国资产阶级的开国元勋、杰出的资产阶级革命家。我觉得，评价克伦威尔应从总体上观察他是否顺应了历史发展的要求，不能简单地因为克伦威尔谋求私利、权欲大、征

In [1]:
# 处理原始jsonl数据到目标格式
import os

train_dataset_path = os.path.join(os.getcwd(), "dataset", "train.jsonl")
test_dataset_path = os.path.join(os.getcwd(), "dataset", "test.jsonl")
train_saved_path = os.path.join(os.getcwd(), "dataset", "new_train.jsonl")
test_saved_path = os.path.join(os.getcwd(), "dataset", "new_test.jsonl")

In [2]:
import json

# 读取jsonl的数据，将text和category作为input，output作为output，再加上prompt
def jsonl_dataset_transfer(old_path, new_path):
    # 处理原来的jsonl文件 & 临时存储结果
    messages = []

    with open(old_path, "r") as f:
        for line in f:
            # 与load进行区分
            data = json.loads(line)
            context = data["text"]
            category = data["category"]
            label = data["output"]

            message = {
                "instruction": "你是文本分类领域的专家，我将会给你一段文本和几个潜在的文本分类选项，请根据文本内容输出正确的文本类型。",
                "input": "文本:{}, 文本分类选项:{}".format(context, category),
                "output": label
            }

            messages.append(message)

    # 将结果写入目标地址进行存储
    with open(new_path, "w", encoding="utf-8") as f:
        for message in messages:
            f.write(json.dumps(message, ensure_ascii=False) + "\n")

In [3]:
jsonl_dataset_transfer(train_dataset_path, train_saved_path)
jsonl_dataset_transfer(test_dataset_path, test_saved_path)

In [4]:
# 上一步是将数据集调整成所需格式，这一步进行处理函数的编写
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

d:\miniconda\envs\transformer\lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [12]:
[tokenizer.eos_token_id]

[151645]

In [19]:
def process_fn(example):
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []

    # Qwen2使用的ChatML格式
    instruction = tokenizer(
        f"<|im_start|>system\n{example['instruction']}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False         # 已经手动写好ChatML的格式，无需特殊字符补充
    )
    response = tokenizer(
        f"{example['output']}",
        add_special_tokens=False
    )

    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    # 此处末尾的[1]标记eos_token也是有效的，帮助模型学习什么时候停止
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]   
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]

    # 观察之前的数据可知，384的长度肯定不够，需要进行截断
    # todo 可以尝试使用滑窗
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH-1] + [input_ids[-1]]
        attention_mask = attention_mask[:MAX_LENGTH-1] + [attention_mask[-1]]
        labels = labels[:MAX_LENGTH-1] + [labels[-1]]
    
    return{
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [20]:
# 读取数据
from datasets import Dataset

json_path = train_dataset_path = os.path.join(os.getcwd(), "dataset", "new_train.jsonl")
dataset = Dataset.from_json(json_path)
tokenized_dataset = dataset.map(process_fn, remove_columns=dataset.column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

# 加载Qwen模型

In [34]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

# 配置Lora

In [35]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [36]:
model = get_peft_model(model, config)

In [24]:
model.print_trainable_parameters()

trainable params: 9,232,384 || all params: 1,552,946,688 || trainable%: 0.5945


# 配置TrainingAruguments & 创建Trainer

In [37]:
args = TrainingArguments(
    output_dir="./weight",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=2,
    metric_for_best_model="accuracy",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

# 模型训练

In [38]:
# os.environ["SWANLAB_DISABLE"] = "true"

trainer.train()

Step,Training Loss
10,16.845500
20,8.726800
30,0.217100
40,0.140900
50,0.076400
60,0.075900
70,0.032700
80,0.106300
90,0.045400
100,0.168400


TrainOutput(global_step=500, training_loss=0.5477490829369053, metrics={'train_runtime': 560.1981, 'train_samples_per_second': 14.281, 'train_steps_per_second': 0.893, 'total_flos': 2.4322369388544e+16, 'train_loss': 0.5477490829369053, 'epoch': 2.0})